In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
spam_df=pd.read_csv('../Downloads/spam.csv',encoding = "ISO-8859-1")

In [3]:
x_samp=spam_df.sample(10)


In [4]:
x_samp['v2']=x_samp['v2'].str.lower()

In [5]:
x_samp

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
2012,ham,beautiful truth against gravity.. read careful...,NaN,NaN,NaN
4644,ham,are you planning to come chennai?,NaN,NaN,NaN
5200,spam,call germany for only 1 pence per minute! call...,NaN,NaN,NaN
3921,ham,"oh really? perform, write a paper, go to a mov...",NaN,NaN,NaN
153,ham,as per your request 'melle melle (oru minnamin...,NaN,NaN,NaN
2219,spam,you have won a guaranteed å£1000 cash or a å£2...,NaN,NaN,NaN
2190,ham,"ooh, 4got, i'm gonna start belly dancing in mo...",NaN,NaN,NaN
3393,ham,bull. your plan was to go floating off to ikea...,NaN,NaN,NaN
1287,ham,"what do you do, my dog ? must i always wait ti...",NaN,NaN,NaN
3111,ham,just haven't decided where yet eh ?,NaN,NaN,NaN


In [6]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [7]:
cleanhtml('<h1>Ernest harnesser</h1><br/><a href="we.ho">link</a>')

'Ernest harnesserlink'

In [8]:
import string
exclude=string.punctuation
def punc_rem(text):
    return text.translate(str.maketrans('','',exclude)).strip()


In [9]:
punc_rem('Hello ! hey there,willl be there\' ?')

'Hello  hey therewilll be there'

In [10]:
x_samp['v2'].apply(punc_rem)

2012    beautiful truth against gravity read carefully...
4644                     are you planning to come chennai
5200    call germany for only 1 pence per minute call ...
3921    oh really perform write a paper go to a movie ...
153     as per your request melle melle oru minnaminun...
2219    you have won a guaranteed å£1000 cash or a å£2...
2190    ooh 4got im gonna start belly dancing in mosel...
3393    bull your plan was to go floating off to ikea ...
1287    what do you do my dog  must i always wait till...
3111                     just havent decided where yet eh
Name: v2, dtype: object

In [11]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [12]:
def chat_word_adjuster(text):
    word_list=[]
    text=text.lower()
    for word in text.split():
        if word in abbreviations:
            word_list.append(abbreviations[word])
        else:
            word_list.append(word)
    return " ".join(word_list)       

In [13]:
chat_word_adjuster('u are')

'you are'

In [14]:
from textblob import TextBlob

In [17]:
def word_text_correction(text):
    txt=TextBlob(text)
    return txt.correct().string

In [25]:
word_text_correction('corrct note book is notebook')

'correct note book is notebook'

In [28]:
from nltk.corpus import stopwords
stp_wrds=stopwords.words('english')

In [46]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ujwal\AppData\Roaming\nltk_data...


True

In [35]:
from nltk.tokenize import word_tokenize,sent_tokenize
text_="I am avail at ujex.com.gmaili"
word_tokenize(text_)

['I', 'am', 'avail', 'at', 'ujex.com.gmaili']

In [42]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem_word(sample):
    for word in sample.split():
        print(ps.stem(word))

In [43]:
stem_word("Walking walks slowly")

walk
walk
slowli


In [44]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [60]:
wnl.lemmatize('walked',pos="v")

'walk'

In [80]:
import requests

url = "https://moviedatabase8.p.rapidapi.com/Search/Lucifer"

headers = {
	"x-rapidapi-key": "16a3b9a93emsh6966766774b8107p14ed55jsnade741b60791",
	"x-rapidapi-host": "moviedatabase8.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
movie_df=response.json()

In [81]:
movie_df[1]

{'_id': '65ef4057675dde8010882fe8',
 'id': 452717,
 'title': 'Hollywood Lucifer',
 'vote_average': 0.8,
 'vote_count': 2,
 'status': 'Released',
 'release_date': '2014-01-22T00:00:00.000Z',
 'revenue': 0,
 'runtime': 3,
 'adult': False,
 'backdrop_path': '/8iNwdrRZib7WsBSNVkvA9nUHGgu.jpg',
 'budget': 0,
 'imdb_id': 'tt12176474',
 'original_language': 'en',
 'original_title': 'Hollywood Lucifer',
 'popularity': 3.189,
 'poster_path': 'https://image.tmdb.org/t/p/original//5ZegOrfxjUfVStdBtOKtLhB2yQQ.jpg',
 'genres': 'Drama, Fantasy',
 'production_countries': 'United States of America',
 'spoken_languages': 'No Language'}

In [82]:
movie_df=pd.DataFrame(movie_df)
movie_df.drop(columns=['_id'],inplace=True)

In [83]:
movie_df

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,homepage
0,564701,Lucifer,7.400,46,Released,2019-03-28T00:00:00.000Z,28717000,174,False,/8SFdD1gwynrsj3hCtINgC1EsNzB.jpg,...,ലൂസിഫെർ,"In God's own country, the supreme leader of th...",7.318,https://image.tmdb.org/t/p/original//fXgY2RCzo...,Blood . Brotherhood . Betrayal,"Action, Drama, Thriller",Aashirvad Cinemas,India,"English, Hindi, Malayalam, Tamil",NaN
1,452717,Hollywood Lucifer,0.800,2,Released,2014-01-22T00:00:00.000Z,0,3,False,/8iNwdrRZib7WsBSNVkvA9nUHGgu.jpg,...,Hollywood Lucifer,NaN,3.189,https://image.tmdb.org/t/p/original//5ZegOrfxj...,NaN,"Drama, Fantasy",NaN,United States of America,No Language,NaN
2,1227405,Lucifer Sam,0.000,0,Released,2018-05-05T00:00:00.000Z,0,3,False,NaN,...,Lucifer Sam,A two-headed Siamese cat chases a mouse trough...,0.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,256298,Lucifer,7.800,2,Released,2007-04-11T00:00:00.000Z,0,8,False,NaN,...,Lucifer,"This is the story of the fall of Lucifer, whos...",2.958,https://image.tmdb.org/t/p/original//qtwN1f39H...,NaN,Adventure,NaN,NaN,NaN,http://www.luciferthemovie.com/
4,635241,Lucifer 3,0.000,0,Planned,NaN,0,0,False,NaN,...,Lucifer 3,The third film in the Lucifer trilogy.,0.805,NaN,NaN,NaN,Aashirvad Cinemas,NaN,Malayalam,NaN
5,299001,Lucifer,5.923,13,Released,2014-10-22T00:00:00.000Z,0,108,False,NaN,...,Lucifer,"On his downfall from Heaven to Hell, Lucifer p...",1.048,https://image.tmdb.org/t/p/original//kMKd5M0ZY...,NaN,Drama,Minds Meet,"Mexico, Belgium",Spanish,http://www.luciferthefilm.com/
6,368119,Lucifer Sensommer - gult og sort,2.000,1,Released,1990-11-18T00:00:00.000Z,0,93,False,NaN,...,Lucifer Sensommer - gult og sort,Lucifer Sensommer Gult og Sort is a Norwegian ...,0.620,https://image.tmdb.org/t/p/original//9XRqSJ8EG...,NaN,NaN,NaN,Norway,Norwegian,NaN
7,1101145,Brides of Lucifer,0.000,0,Released,1972-01-01T00:00:00.000Z,0,56,True,NaN,...,Brides of Lucifer,A distraught husband dabbles in Satanism to lu...,0.000,https://image.tmdb.org/t/p/original//g9m51c3dA...,NaN,Horror,NaN,United States of America,NaN,NaN
8,317902,Island of Lucifer,6.500,1,Released,2012-01-01T00:00:00.000Z,0,0,False,NaN,...,Island of Lucifer,A mysterious and secretive satanic cult has be...,0.600,https://image.tmdb.org/t/p/original//tl7SGMH77...,NaN,NaN,NaN,NaN,NaN,NaN
9,605224,Lucifer,10.000,1,Released,2019-06-06T00:00:00.000Z,0,27,False,/aXiSAUCIu04Hf2atRkPxT65tOdY.jpg,...,Люцифер,Popular Russian rapper Husky experiences an ex...,0.600,https://image.tmdb.org/t/p/original//z0zd89qGI...,NaN,Documentary,Stereotactic,Russia,Russian,NaN


In [85]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
df={
    'text':["hello ram ji","hello shyam ji","hello ujex"],
    'sent':[1,1,0]
}
df=pd.DataFrame(df)

In [86]:
df

,text,sent
0,hello ram ji,1
1,hello shyam ji,1
2,hello ujex,0


In [92]:
bow_=cv.fit_transform(df['text'])

In [93]:
cv.vocabulary_

{'hello': 0, 'ram': 2, 'ji': 1, 'shyam': 3, 'ujex': 4}

In [96]:
print(bow_[0].toarray())
print(bow_[1].toarray())

[[1 1 1 0 0]]
[[1 1 0 1 0]]
